In [ ]:
!pip install openai numpy pandas

In [2]:
import json
import numpy as np
import pandas as pd

In [3]:
from openai import OpenAI
OPENAI_API_KEY= 'YOUR_OPENAI_KEY'
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
client.files.create(
    file=open('/content/scents_training_data - NM edits.jsonl', 'rb'),
    purpose = "fine-tune"
)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-Cj23Myxa5YmEipf3gRxdupfr",
  model="gpt-3.5-turbo"
)

In [ ]:
client.fine_tuning.jobs.retrieve('ftjob-fhtTTQfsEmrR2Ado92iJRZSr')

In [10]:
job_id = 'ftjob-fhtTTQfsEmrR2Ado92iJRZSr'

In [ ]:
import signal
import datetime


def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

In [ ]:
import time

status = client.fine_tuning.jobs.retrieve('ftjob-fhtTTQfsEmrR2Ado92iJRZSr').status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve('ftjob-fhtTTQfsEmrR2Ado92iJRZSr').status
        print(f"Status: {status}")
else:
    print(f"Finetune job ftjob-fhtTTQfsEmrR2Ado92iJRZSr finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

In [ ]:
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)